In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()

%load_ext autoreload
%autoreload 2

import pystan

In [2]:
log_dir = 'log/fabian_sim/EZ/20210912_182221_ezsim_m7/'
# log_dir = 'log/fabian_sim/CL/20210829_134725_sim_m7/'

data = load_obj('data', log_dir)
data.raw_data

{'random_seed': 0,
 'N': 200,
 'K': 2,
 'J': 6,
 'alpha': array([0., 0., 0., 0., 0., 0.]),
 'beta': array([[1. , 0. ],
        [0.8, 0. ],
        [0.8, 0. ],
        [0. , 1. ],
        [0. , 0.8],
        [0. , 0.8]]),
 'sigma_z': array([0.80622577, 0.80622577]),
 'Phi_corr': array([[1. , 0.2],
        [0.2, 1. ]]),
 'Phi_cov': array([[0.65, 0.13],
        [0.13, 0.65]]),
 'Marg_cov': array([[1.    , 0.52  , 0.52  , 0.13  , 0.104 , 0.104 ],
        [0.52  , 1.    , 0.416 , 0.104 , 0.0832, 0.0832],
        [0.52  , 0.416 , 1.    , 0.104 , 0.0832, 0.0832],
        [0.13  , 0.104 , 0.104 , 1.    , 0.52  , 0.52  ],
        [0.104 , 0.0832, 0.0832, 0.52  , 1.    , 0.416 ],
        [0.104 , 0.0832, 0.0832, 0.52  , 0.416 , 1.    ]]),
 'Theta': array([[0.35 , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.584, 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.584, 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.35 , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.584, 

## Run MCMC

In [3]:

# model_num = 7

# sm = load_obj('sm', 'log/compiled_models/model'+str(model_num)+'/')

# param_names = model_phonebook(model_num)['param_names']
# latent_names = model_phonebook(model_num)['latent_names']
# stan_names = model_phonebook(model_num)['stan_names']

# fit_run = run_mcmc(
#     data.get_stan_data(),
#     sm,
#     num_samples = 1000,
#     num_warmup = 1000,
#     adapt_engaged=True,
#     num_chains = 1,
#     log_dir = './log/20210725_224158_debug/'
# )
# ps = fit_run.extract(permuted=False, pars=param_names)
# save_obj(ps, 'mcmcps', 'log/fabian_sim/EZ/20210912_182221_ezsim_m7/')

In [4]:
ps = load_obj('mcmcps', 'log/fabian_sim/EZ/20210912_182221_ezsim_m7/')

## Run IBIS

In [5]:
# load existing results

ibis = load_obj('particles', log_dir)


In [6]:
gen_model = 0
model_num = 1


for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(ibis.particles[name])
    w = exp_and_normalise(ibis.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))



Estimate
[-0.06 -0.01  0.    0.02  0.05 -0.03]


Estimate
[[0.94 0.52 0.49 0.04 0.03 0.03]
 [0.52 0.91 0.42 0.03 0.02 0.03]
 [0.49 0.42 1.01 0.03 0.02 0.02]
 [0.04 0.03 0.03 0.99 0.49 0.51]
 [0.03 0.02 0.02 0.49 0.92 0.41]
 [0.03 0.03 0.02 0.51 0.41 1.01]]


## Plot Both results

In [7]:
# have to resample particles to get rid of weights
ibis.resample_particles()
particles = ibis.particles


In [113]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'MCMC'
df2 = get_post_df(particles[param])
df2['source'] = 'IBIS'
plot_data = pd.concat([df,df2])
plot_data['row'] =plot_data['row'] + 1
plot_data['col'] =plot_data['col'] + 1

plot_density(plot_data).configure_legend(
    titleFontSize=16,
    labelFontSize=14,
    symbolStrokeWidth=6
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
)

alt.Chart(...)

In [112]:
param = 'alpha'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'MCMC'
df2 = get_post_df(particles[param])
df2['source'] = 'IBIS'

plot_data = pd.concat([df,df2])
plot_data['row'] = plot_data['col']+1
plot_data['col'] = 1

plot_density(plot_data).configure_legend(
    titleFontSize=16,
    labelFontSize=14,
    symbolStrokeWidth=6
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=0
)

alt.Chart(...)

In [111]:
param = 'Phi_cov'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'MCMC'
df2 = get_post_df(particles[param])
df2['source'] = 'IBIS'

plot_data = pd.concat([df,df2])
plot_data['row'] =plot_data['row'] + 1
plot_data['col'] =plot_data['col'] + 1

plot_density(plot_data).configure_legend(
    titleFontSize=16,
    labelFontSize=14,
    symbolStrokeWidth=6
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=16
)

alt.Chart(...)

### credible intervals

In [76]:
param = 'beta'
df = get_post_df(particles[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])
df['row'] = df['row']+1
df['col'] = df['col']+1

dd = pd.DataFrame(data.raw_data['beta'], columns=['1', '2'])
dd['row'] = np.arange(1, dd.shape[0]+1)
dd = dd.melt(id_vars='row', var_name='col', value_name = 'data')
dd['col'] = dd.col.astype(int)

plot_data = df.merge(dd, on=['row', 'col'])
plot_data['index'] = 'Loading(' + plot_data.row.astype(str)+','+plot_data.col.astype(str)+')'

c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None))
c1



c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None)
)


(c1+c2).facet(
    facet=alt.Facet('index:N', title=None),
    title='',
    columns=2
).configure_axis(
    labelFontSize=14,
).configure_title(
    fontSize=16
).configure_header(labelFontSize=16)


alt.FacetChart(...)

In [82]:
param = 'alpha'
df = get_post_df(particles[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])
df['data'] = 0
plot_data = df
plot_data['index'] = 'Intercept('+plot_data.col.astype(str)+')'


c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None))
c1



c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None)
)


(c1+c2).facet(
    facet=alt.Facet('index:N', title=None),
    title='',
    columns=2
).configure_axis(
    labelFontSize=14,
).configure_title(
    fontSize=16
).configure_header(labelFontSize=16)


alt.FacetChart(...)

In [79]:
data.raw_data['Phi_cov']

array([[0.65, 0.13],
       [0.13, 0.65]])

In [81]:
param = 'Phi_cov'
df = get_post_df(particles[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])
df['row'] = df['row']+1
df['col'] = df['col']+1

dd = pd.DataFrame(data.raw_data['Phi_cov'], columns=['1', '2'])
dd['row'] = np.arange(1, dd.shape[0]+1)
dd = dd.melt(id_vars='row', var_name='col', value_name = 'data')
dd['col'] = dd.col.astype(int)

plot_data = df.merge(dd, on=['row', 'col'])
plot_data['index'] = 'Factor Cov(' + plot_data.row.astype(str)+','+plot_data.col.astype(str)+')'

c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None))
c1



c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None)
)


(c1+c2).facet(
    facet=alt.Facet('index:N', title=None),
    title='',
    columns=2
).configure_axis(
    labelFontSize=14,
).configure_title(
    fontSize=16
).configure_header(labelFontSize=16)


alt.FacetChart(...)